In [1]:
import pandas as pd
import cvxpy as cp
import numpy as np
from pathlib import Path

In [2]:
FILE_NAME = 'work_planning_example.xlsx'

data = pd.read_excel(FILE_NAME, index_col=0)
data

,GI,Liver,HPB,Breast,DCC PAs
Wonderwoman,4,NaN,x,NaN,9.0
Superman,x,NaN,<3,NaN,8.5
The Joker,NaN,>5,x,NaN,10.0
Catwoman,x,NaN,NaN,x,6.0
Emergency cover,4,2,4,2,NaN
Estimated demand,12,6,12,4,NaN


The optimization problem to model the resource allocation tasks is as follows
$$
\begin{alignedat}{3}
& \min_{x_{c,s}, \delta_s, \gamma_s, \iota_c, t_s} & \quad & \sum_{s \in \mathcal{S}} \delta_s + \sum_{c \in \mathcal{C}} \iota_c + 2 \sum_{s \in \mathcal{S}} \gamma_s \\
& \text{subject to} &&& t_s &= \sum_{c \in \mathcal{C}_s} x_{c,s} & \quad & \forall s \in \mathcal{S}, \\
&&&& t_s + \gamma_s &\geq \mathrm{PA}_{\min,s} && \forall s \in \mathcal{S}, \\
&&&& t_s + \gamma_s + \delta_s &= \mathrm{PA}_{\mathrm{est},s} && \forall s \in \mathcal{S}, \\
&&&& \sum_{s \in \mathcal{S}_c} x_{c,s} + \iota_c &= \mathrm{DCC}_c && \forall c \in \mathcal{C}, \\
&&&& x_{c, s} & \geq 0.5 && \forall s \in \mathcal{S}_c, \quad \forall c \in \mathcal{C}, \\
&&&& x_{c, s} & \in \mathcal{J}_{c,s} && \forall s \in \mathcal{S}_c, \quad \forall c \in \mathcal{C}, \\
&&&& \gamma_s, \delta_s, \iota_c & \geq 0 && \forall s \in \mathcal{S}, c \in \mathcal{C}. 
\end{alignedat}
$$

The sets $\mathcal{C}$ and $\mathcal{S}$ are the sets of all consultants and all specialities respectively. Their subscripted counterparts $\mathcal{C}_s$ and $\mathcal{S}_c$ are all the consultants that are able to work on Speciality $s$ and its counterpart, all specialities that the Consultant $c$ can work on respectively. 

The $\iota_s$ are the missing PAs that are necessary to cover the minimum workload and emergencies in Speciality $s$. Any non-zero $\iota_s$ is critical and impedes patient safety as minimal coverage _cannot_ be guaranteed.
$\delta_s$ is the difference between the estimated workload in Speciality $s$ and the resources that were allocated. Finally, the variable $\iota_c$ captures the eventual time a particular Consultant $c$ is idle.

The main optimization variables are the $x_{c,s}$s, the allocated PAs for Consultant $c$ in Speciality $s$. The auxiliary variable $t_s$ is introduced for better legibiliy and contains the PAs covered in Speciality $s$ by all available consultants.

The constraint $x_{c,s} \geq 0.5$ ensures that if a consutant is working in a speciality, this consultant keeps a minimum workload to not loose practice. The constraint-sets $\mathcal{J}_{c,s}$ capture particular requirements stemming from individual jobplans, such as minimum or maximum contributions of a consultant to a particular speciality, if applicable.

In [3]:
consultants = data.iloc[:-2]
consultants

,GI,Liver,HPB,Breast,DCC PAs
Wonderwoman,4,NaN,x,NaN,9.0
Superman,x,NaN,<3,NaN,8.5
The Joker,NaN,>5,x,NaN,10.0
Catwoman,x,NaN,NaN,x,6.0


In [4]:
specialities = data.iloc[-2:, :-1]
specialities

,GI,Liver,HPB,Breast
Emergency cover,4,2,4,2
Estimated demand,12,6,12,4


In [5]:
reqs = pd.melt(consultants.iloc[:,:-1].reset_index(), id_vars='index').dropna()
reqs.reset_index(drop=True, inplace=True)
reqs.columns = ['consultant', 'speciality', 'availability']
reqs.consultant = reqs.consultant.astype('category')
reqs.availability = reqs.availability.astype('str')
reqs.loc[:, 'consultant_id'] = reqs.consultant.cat.codes
reqs.speciality = reqs.speciality.astype('category')
reqs.loc[:, 'speciality_id'] = reqs.speciality.cat.codes
reqs.head()

,consultant,speciality,availability,consultant_id,speciality_id
0,Wonderwoman,GI,4,3,1
1,Superman,GI,x,1,1
2,Catwoman,GI,x,0,1
3,The Joker,Liver,>5,2,3
4,Wonderwoman,HPB,x,3,2


In [6]:
num_specialities = specialities.shape[1]
t_s = cp.Variable(num_specialities)
γ_s = cp.Variable(num_specialities, nonneg=True)
δ_s = cp.Variable(num_specialities, nonneg=True)

num_consultants = len(consultants)
ι_c = cp.Variable(num_consultants, nonneg=True)

x_cs = cp.Variable(len(reqs))

constr = [x_cs >= .5]

In [7]:
for (speciality_name, speciality_id), rows in reqs.groupby(['speciality', 'speciality_id']):
    min_cover = specialities.loc['Emergency cover', speciality_name]
    est_demand = specialities.loc['Estimated demand', speciality_name]
    
    print(
        speciality_name, ':', [x for x in rows.consultant],
        '=', est_demand, f'(min: {min_cover})'
    )
      
    t = t_s[speciality_id]
    
    constr.append(t == cp.sum(x_cs[rows.index]))
    constr.append(t + γ_s[speciality_id] >= min_cover)
    constr.append(t + γ_s[speciality_id] + δ_s[speciality_id] == est_demand)
   
print()

for (consultant_name, consultant_id), rows in reqs.groupby(['consultant', 'consultant_id']):
    DCCs = consultants.loc[consultant_name, 'DCC PAs']
    
    print(consultant_name, ':', [x for x in rows.speciality], '=', DCCs)
    
    constr.append(
        cp.sum(x_cs[rows.index]) + ι_c[consultant_id] == DCCs)

Breast : ['Catwoman'] = 4 (min: 2)
GI : ['Wonderwoman', 'Superman', 'Catwoman'] = 12 (min: 4)
HPB : ['Wonderwoman', 'Superman', 'The Joker'] = 12 (min: 4)
Liver : ['The Joker'] = 6 (min: 2)

Catwoman : ['GI', 'Breast'] = 6.0
Superman : ['GI', 'HPB'] = 8.5
The Joker : ['Liver', 'HPB'] = 10.0
Wonderwoman : ['GI', 'HPB'] = 9.0


In [8]:
# flexible assignments --> do nothing
idx = reqs.availability == 'x'

# min value
idx = reqs.availability.str.startswith('>')
if idx.sum():
    min_pa = reqs.loc[idx, 'availability'].str[1:].astype('float')
    constr.append(
        x_cs[min_pa.index] >= min_pa.values
    )

# max value
idx = reqs.availability.str.startswith('<')
if idx.sum():
    max_pa = reqs.loc[idx, 'availability'].str[1:].astype('float')
    constr.append(
        x_cs[max_pa.index] <= max_pa.values
    )

# exact value
idx = reqs.availability.str.isnumeric()
if idx.sum():
    exact_pa = reqs.loc[idx, 'availability'].astype('float')
    constr.append(
       x_cs[exact_pa.index] == exact_pa.values 
    )

In [9]:
obj = cp.Minimize(
    2 * cp.sum(γ_s) + cp.sum(δ_s) + cp.sum(ι_c)
)

prob = cp.Problem(obj, constr)
prob.solve(solver=cp.SCS)

0.4999996656206974

In [10]:
df = pd.DataFrame()
for s, c, v in zip(reqs.speciality, reqs.consultant, np.round(x_cs.value, 2)):
    df.loc[c, s] = v

Cs = consultants.index
Ss = specialities.columns
df = df.loc[Cs, Ss]

df.loc[Cs, 'Idle'] = np.round(ι_c.value, 2)
df.loc[Cs, 'DCC PAs'] = consultants.loc[:, 'DCC PAs']
df.loc['sum', :] = df.loc[Cs, :].sum()
df.fillna(0, inplace=True)

df.loc['min cover gap', Ss] = np.round(γ_s.value, 2)
df.loc['demand gap', Ss] = np.round(δ_s.value, 2)
df.loc['Emergency cover', Ss] = specialities.loc['Emergency cover', :]
df.loc['Estimated demand', Ss] = specialities.loc['Estimated demand', :]
df.loc['min cover gap':, 'DCC PAs'] = df.loc['min cover gap':, Ss].sum('columns')

fn = Path(FILE_NAME)
df.to_excel(fn.with_suffix('.result' + fn.suffix))
df

,GI,Liver,HPB,Breast,Idle,DCC PAs
Wonderwoman,4.00,0.00,5.00,0.00,0.0,9.00
Superman,5.75,0.00,2.75,0.00,0.0,8.50
The Joker,0.00,5.90,4.10,0.00,0.0,10.00
Catwoman,2.13,0.00,0.00,3.87,0.0,6.00
sum,11.88,5.90,11.85,3.87,0.0,33.50
min cover gap,0.00,0.00,0.00,0.00,NaN,0.00
demand gap,0.13,0.12,0.16,0.10,NaN,0.51
Emergency cover,4.00,2.00,4.00,2.00,NaN,12.00
Estimated demand,12.00,6.00,12.00,4.00,NaN,34.00
